# Default Rate Sensitivity on Marketing Spend Framework - MX

The effects of changing default rates on Liang & Zu's marketing spend framework is explored.

---

In [120]:
import pandas as pd
import numpy as np
import pyltv
from dbm import DBM
import plotly
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

In [121]:
data = pd.read_csv('data/mx_data.csv')

### Study Assumptions
1. Default rate stress applied as a blanket stress across all time periods.
2. Default rate stress applied to both 7dpd and 365dpd default rates.
3. The default rate stress is simply a percentage point added to the actual default rate. **So if the 365dpd default rate is currently 6%, adding a 1% stress results in a new 365dpd default rate of 7%**.

## Impact of Default Rate on LTV

In [122]:
market='mx'
forecast_method='powerslope'
min_months=4
n_months=120
default_stress=0.01

m1 = pyltv.Model(data, market=market, fcast_method=forecast_method)
m1.generate_features()
m1.forecast = m1.forecast_data(m1.data, min_months, n_months)

m2 = pyltv.Model(data, market=market, fcast_method=forecast_method, default_stress=default_stress)
m2.generate_features()
m2.forecast = m2.forecast_data(m2.data, min_months, n_months)

Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...


In [123]:
colors = {i: c for i, c in enumerate(plotly.colors.qualitative.Dark24)}
# add additional colors
color_size = len(colors)
for i, c in enumerate(plotly.colors.qualitative.Light24):
    colors[i+color_size] = c

fig = make_subplots(rows=1, cols=2, shared_xaxes=True,
                   subplot_titles=('Unstressed', '+1% Default Rates'))

param = 'cumulative_dcf_ltv_per_original'

# LEFT PLOT
curves1=[]
for cohort in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves1.append(output)
    
for i, cohort in enumerate(curves1):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}'), row=1, col=1)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}'), row=1, col=1)

expected = m1.ltv_expected.cumulative_ltv_per_original

fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                         line=dict(color='black', width=3), legendgroup='e'), row=1, col=1)
fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                        line=dict(color='black', width=3), legendgroup='e', showlegend=False), row=1, col=2)

# RIGHT PLOT
curves2=[]
for cohort in m2.forecast.cohort.unique():
    c_data = m2.forecast[m2.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves2.append(output)
        
for i, cohort in enumerate(curves2):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2,)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2)

fig.update_layout(title='DCF LTV')
fig.layout.xaxis.title='Month'
fig.layout.xaxis2.title='Month'
fig.update_yaxes(range=[-10, 95])
fig.update_xaxes(range=[-5,52])

fig.show()

### Standard Default Rate

In [124]:
data_50mo1 = m1.forecast[m1.forecast['Months Since First Loan Disbursed']==50]
ltv1 = data_50mo1['cumulative_ltv_per_original'].median()
dcf_ltv1 = data_50mo1['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv1, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv1, 2)}')

Median DCF 50mo LTV: $62.82


### +1% Default Rate Stress

In [125]:
data_50mo2 = m2.forecast[m2.forecast['Months Since First Loan Disbursed']==50]
ltv2 = data_50mo2['cumulative_ltv_per_original'].median()
dcf_ltv2 = data_50mo2['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv2, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv2, 2)}')

Median DCF 50mo LTV: $52.43


In [126]:
print('Change in DCF LTV:')
print(f'{round(100*(dcf_ltv2-dcf_ltv1)/dcf_ltv1, 2)}%')

Change in DCF LTV:
-16.54%


In [127]:
ltvs = {}
for x in np.linspace(-0.03, 0.03, 7):
    m = pyltv.Model(data, market=market, fcast_method=forecast_method, default_stress=x)
    m.generate_features()
    m.forecast = m.forecast_data(m.data, min_months, n_months)
    
    data_50mo = m.forecast[m.forecast['Months Since First Loan Disbursed']==50]
    m.forecast['defaulted_origination'] = m.forecast['default_rate_365dpd']*m.forecast.origination_per_original
    default = m.forecast.defaulted_origination.sum()/m.forecast.origination_per_original.sum()
    
    ltvs[round(x, 2)] = (round(data_50mo['cumulative_dcf_ltv_per_original'].median(), 2), 
                         round(100*default, 2))

Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...


### Default Rate Stress: -3% to +3%

The plot below shows the relationship between a change in the default rate stress and the median DCF LTV for all cohorts.

In [128]:
v = [i[0] for i in ltvs.values()]
l = [f'{i[1]}%' for i in ltvs.values()]
fig = go.Figure(go.Scatter(x=l, y=v, marker=dict(size=10)))
fig.update_layout(xaxis=dict(title='Default Rate Stress'),
                 yaxis=dict(title='Weighted Average DCF LTV'))
fig.show()

In [129]:
print('For every +1% increase in default rate...')
print(f'-${round(ltvs[0][0]-ltvs[0.01][0], 2)}')

For every +1% increase in default rate...
-$10.39


The relationships between LTV and default rates is shown below and explains the linear trend. **Note that this analysis does not include the effects of default rate changes on retention.**

LTV ~ [revenue - (origination+revenue)*default_rate_365dpd]

revenue ~ [0.08 * default_rate_7dpd]

## Default Rate Historicals

#### Weighted Average Lifetime Default

This method uses the average lifetime default rate (365dpd) for each cohort. The average lifetime default rate is a weighted average of defaults by origination_per_original.

In [130]:
defaults = []
default_changes = []
avg_lifetime_default = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c][['origination_per_original', 'default_rate_365dpd']]
    c_data['defaulted_origination'] = c_data['default_rate_365dpd']*c_data.origination_per_original
    c_data['defaulted_origination_change'] = (c_data.defaulted_origination-c_data.defaulted_origination.shift(1))/ \
        c_data.defaulted_origination
    
    defaults.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination))
    default_changes.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination_change))
    
    avg_lifetime_default.append(c_data.defaulted_origination.sum()/c_data.origination_per_original.sum())
    
avg_lifetime_default = pd.Series(avg_lifetime_default, index=m1.forecast.cohort.unique())

In [131]:
avg_lifetime_default.mean()

0.09090640727070719

In [132]:
avg_lifetime_default.std()

0.00867158577545118

In [133]:
10.68-9.25

1.4299999999999997

In [134]:
fig=go.Figure(defaults)
fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Defaulted Origination per Original'))
fig.show()

In [135]:
fig = go.Figure(
    go.Scatter(name=c, x=avg_lifetime_default.index, y=avg_lifetime_default, mode='markers',
                              marker=dict(size=15))
)

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

In MX, the default rates are much higher on average. The median weighted average default rate is 9.1%. There is some apparent seasonality in the default rates that's not seen in the other markets. **We see a maximum weighted average default rate of 10.7% for MX**.

#### Unweighted Average Default

In this method, we simply look at aggregate stats of the 365 dpd default rate, without taking into account origination. This scenario looks at a blanket default rate increase across the board, regardless of the size of loans.

In [136]:
default_stats = {}
for i, c in enumerate(m1.forecast.cohort.unique()):
    c_data = m1.forecast[m1.forecast.cohort==c][['default_rate_365dpd']]
    
    default_stats[c] = (float(c_data.median()), float(c_data.quantile(0.9)))

In [137]:
medians=[]
quantile90=[]
for c in default_stats:
    medians.append(default_stats[c][0])
    quantile90.append(default_stats[c][1])
    
medians = pd.Series(medians, index=np.arange(1, len(medians)+1))
#params1, covs1 = curve_fit(reg, xdata=medians.index, ydata=medians)

quantile90 = pd.Series(quantile90, index=np.arange(1, len(medians)+1))
#params2, covs2 = curve_fit(reg, xdata=quantile90.index, ydata=quantile90)
    
fig = go.Figure([
    go.Scatter(name='median default', x=medians.index, y=medians, mode='markers'),
    #go.Scatter(name='median-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params1[0], params1[1])),
    go.Scatter(name='90 percentile', x=quantile90.index, y=quantile90, mode='markers')
    #go.Scatter(name='90 percentile-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params2[0], params2[1]))
])

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

Similarly, in looking at both the 50% and 90% quantiles of the 365dpd default rates, the trend is quite flat with some similar seasonality as the weighted average default rates. The max we see here for MX is 12.45%.

## Payback Period

### mCAC Payback Period

In [138]:
ratios = [1.25, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [139]:
ratios = [1.25, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The table below shows the payback periods for unstressed default rates in the 1.5x scenario.

In [140]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,mCAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,64.714451,43.142967,22,44.419680
1,2020-10,62.961480,41.974320,22,42.164816
2,2020-11,53.308509,35.539006,23,35.568105
3,2020-12,38.627519,25.751679,25,26.088840
4,2021-01,34.478095,22.985396,26,23.430194
5,2021-02,35.404057,23.602705,26,23.978552
6,2021-03,46.168138,30.778758,25,31.202622
7,2021-04,62.670081,41.780054,24,42.197017
8,2021-05,85.601268,57.067512,24,58.788816
9,2021-06,83.418410,55.612273,24,57.021567


The second table shows the new payback periods in the same 1.5x scenario, but with a +1% default rate stress applied. The payback periods are pushed out by several months.

In [141]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,DCF LTV Stressed,mCAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,64.714451,54.045910,43.142967,29,43.598033
1,2020-10,62.961480,52.572427,41.974320,30,42.690037
2,2020-11,53.308509,43.451457,35.539006,32,35.893873
3,2020-12,38.627519,30.153727,25.751679,36,25.963796
4,2021-01,34.478095,26.463742,22.985396,38,23.263816
5,2021-02,35.404057,27.415382,23.602705,37,23.712899
6,2021-03,46.168138,37.322911,30.778758,34,31.169224
7,2021-04,62.670081,52.282341,41.780054,31,41.971052
8,2021-05,85.601268,73.088622,57.067512,30,58.321070
9,2021-06,83.418410,70.982331,55.612273,30,56.463477


In [143]:
fig1 = go.Figure([go.Scatter(name='1.25x', x=payback_results1['Payback Period (1.25)'], 
                            y=payback_results1['Payback Amount (1.25)'], mode='markers',
                            marker=dict(color='red')),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[12, 40]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1.25x', x=payback_results2['Payback Period (1.25)'], 
                            y=payback_results2['Payback Amount (1.25)'], mode='markers',
                            marker=dict(color='red')),
                  go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig2.update_layout(title='+1% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[12, 40]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [144]:
payback_results1

,cohort,DCF LTV,mCAC (1.25),CAC (1.25),Payback Period (1.25),Payback Amount (1.25),mCAC (1.5),CAC (1.5),Payback Period (1.5),Payback Amount (1.5),mCAC (2),CAC (2),Payback Period (2),Payback Amount (2)
0,2020-09,64.714451,51.771561,14.674479,28,51.783298,43.142967,17.349719,22,44.419680,32.357226,14.674479,15,32.773584
1,2020-10,62.961480,50.369184,14.276980,29,50.743944,41.974320,16.879753,22,42.164816,31.480740,14.276980,16,32.115044
2,2020-11,53.308509,42.646807,12.088097,30,42.988973,35.539006,14.291825,23,35.568105,26.654254,12.088097,17,26.826660
3,2020-12,38.627519,30.902015,8.759075,32,31.521245,25.751679,10.355903,25,26.088840,19.313759,8.759075,19,19.586079
4,2021-01,34.478095,27.582476,7.818162,32,27.722546,22.985396,9.243457,26,23.430194,17.239047,7.818162,20,17.559626
5,2021-02,35.404057,28.323246,8.028131,32,28.420186,23.602705,9.491704,26,23.978552,17.702029,8.028131,20,17.899019
6,2021-03,46.168138,36.934510,10.468965,32,37.698101,30.778758,12.377517,25,31.202622,23.084069,10.468965,19,23.397467
7,2021-04,62.670081,50.136065,14.210903,31,50.940261,41.780054,16.801630,24,42.197017,31.335041,14.210903,18,31.661882
8,2021-05,85.601268,68.481015,19.410718,30,68.902196,57.067512,22.949402,24,58.788816,42.800634,19.410718,18,44.847370
9,2021-06,83.418410,66.734728,18.915739,30,66.966068,55.612273,22.364185,24,57.021567,41.709205,18.915739,18,43.338971


In [145]:
payback_results2

,cohort,DCF LTV,DCF LTV Stressed,mCAC (1.25),CAC (1.25),Payback Period (1.25),Payback Amount (1.25),mCAC (1.5),CAC (1.5),Payback Period (1.5),Payback Amount (1.5),mCAC (2),CAC (2),Payback Period (2),Payback Amount (2)
0,2020-09,64.714451,54.045910,51.771561,14.674479,43,51.873580,43.142967,17.349719,29,43.598033,32.357226,14.674479,20,33.616923
1,2020-10,62.961480,52.572427,50.369184,14.276980,44,50.716572,41.974320,16.879753,30,42.690037,31.480740,14.276980,20,31.565012
2,2020-11,53.308509,43.451457,42.646807,12.088097,47,42.677811,35.539006,14.291825,32,35.893873,26.654254,12.088097,22,26.846580
3,2020-12,38.627519,30.153727,30.902015,8.759075,55,30.992243,25.751679,10.355903,36,25.963796,19.313759,8.759075,26,20.050162
4,2021-01,34.478095,26.463742,27.582476,7.818162,58,27.642774,22.985396,9.243457,38,23.263816,17.239047,7.818162,27,17.570844
5,2021-02,35.404057,27.415382,28.323246,8.028131,56,28.379350,23.602705,9.491704,37,23.712899,17.702029,8.028131,27,18.217961
6,2021-03,46.168138,37.322911,36.934510,10.468965,49,37.084309,30.778758,12.377517,34,31.169224,23.084069,10.468965,24,23.207047
7,2021-04,62.670081,52.282341,50.136065,14.210903,44,50.170415,41.780054,16.801630,31,41.971052,31.335041,14.210903,22,31.483264
8,2021-05,85.601268,73.088622,68.481015,19.410718,41,68.526722,57.067512,22.949402,30,58.321070,42.800634,19.410718,21,43.638014
9,2021-06,83.418410,70.982331,66.734728,18.915739,42,67.117921,55.612273,22.364185,30,56.463477,41.709205,18.915739,21,42.066751


In [146]:
payback_results1['Payback Period (1.25)'].mean()

30.666666666666668

In the worst case, the **payback period will shift out 11 months at an LTV/mCAC of 1.5**, and by **6 months for an LTV/mCAC of 2**.

### Average CAC Payback Period

In [111]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [112]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The two tables below show the changes in payback periods when using LTV/CAC instead of LTV/mCAC as the guard rail. The shifts in payback periods are much less significant.

In [113]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,CAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,64.714451,17.349719,8,17.362731
1,2020-10,62.961480,16.879753,9,17.151570
2,2020-11,53.308509,14.291825,11,15.581920
3,2020-12,38.627519,10.355903,13,11.484720
4,2021-01,34.478095,9.243457,14,10.125682
5,2021-02,35.404057,9.491704,14,10.190310
6,2021-03,46.168138,12.377517,13,13.571406
7,2021-04,62.670081,16.801630,12,18.695580
8,2021-05,85.601268,22.949402,11,24.349631
9,2021-06,83.418410,22.364185,11,23.317359


In [114]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,DCF LTV Stressed,CAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,64.714451,49.452833,17.349719,12,17.356573
1,2020-10,62.961480,48.099677,16.879753,13,17.380097
2,2020-11,53.308509,39.207753,14.291825,15,14.948706
3,2020-12,38.627519,26.505556,10.355903,19,11.057503
4,2021-01,34.478095,23.013375,9.243457,20,9.276664
5,2021-02,35.404057,23.976068,9.491704,20,9.758460
6,2021-03,46.168138,33.514825,12.377517,18,13.513293
7,2021-04,62.670081,47.810157,16.801630,15,17.103238
8,2021-05,85.601268,67.701602,22.949402,14,24.320720
9,2021-06,83.418410,65.628277,22.364185,14,23.180098


In [115]:
fig1 = go.Figure([go.Scatter(name='1x', x=payback_results1['Payback Period (1)'], 
                            y=payback_results1['Payback Amount (1)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers')
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[5, 23]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1x', x=payback_results2['Payback Period (1)'], 
                            y=payback_results2['Payback Amount (1)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers')
                ])

fig2.update_layout(title='+1% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[5, 23]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [116]:
payback_results1

,cohort,DCF LTV,mCAC (1),CAC (1),Payback Period (1),Payback Amount (1),mCAC (1.5),CAC (1.5),Payback Period (1.5),Payback Amount (1.5),mCAC (2),CAC (2),Payback Period (2),Payback Amount (2)
0,2020-09,64.714451,64.714451,23.704927,11,23.748814,43.142967,17.349719,8,17.362731,32.357226,14.674479,7,15.082753
1,2020-10,62.961480,62.961480,23.062813,12,23.283235,41.974320,16.879753,9,17.151570,31.480740,14.276980,8,14.882252
2,2020-11,53.308509,53.308509,19.526926,13,19.943339,35.539006,14.291825,11,15.581920,26.654254,12.088097,10,13.671723
3,2020-12,38.627519,38.627519,14.149274,15,14.336638,25.751679,10.355903,13,11.484720,19.313759,8.759075,12,10.053876
4,2021-01,34.478095,34.478095,12.629339,16,12.752578,22.985396,9.243457,14,10.125682,17.239047,7.818162,13,8.766456
5,2021-02,35.404057,35.404057,12.968519,17,14.206252,23.602705,9.491704,14,10.190310,17.702029,8.028131,13,8.779747
6,2021-03,46.168138,46.168138,16.911406,15,17.043201,30.778758,12.377517,13,13.571406,23.084069,10.468965,12,11.825415
7,2021-04,62.670081,62.670081,22.956074,14,23.239332,41.780054,16.801630,12,18.695580,31.335041,14.210903,11,16.398825
8,2021-05,85.601268,85.601268,31.355776,14,33.573937,57.067512,22.949402,11,24.349631,42.800634,19.410718,10,21.383425
9,2021-06,83.418410,83.418410,30.556194,14,32.314580,55.612273,22.364185,11,23.317359,41.709205,18.915739,10,20.432451


In [117]:
payback_results2

,cohort,DCF LTV,DCF LTV Stressed,mCAC (1),CAC (1),Payback Period (1),Payback Amount (1),mCAC (1.5),CAC (1.5),Payback Period (1.5),Payback Amount (1.5),mCAC (2),CAC (2),Payback Period (2),Payback Amount (2)
0,2020-09,64.714451,49.452833,64.714451,23.704927,16,24.714422,43.142967,17.349719,12,17.356573,32.357226,14.674479,11,15.887022
1,2020-10,62.961480,48.099677,62.961480,23.062813,17,24.085049,41.974320,16.879753,13,17.380097,31.480740,14.276980,12,15.308933
2,2020-11,53.308509,39.207753,53.308509,19.526926,19,20.161893,35.539006,14.291825,15,14.948706,26.654254,12.088097,13,12.131451
3,2020-12,38.627519,26.505556,38.627519,14.149274,23,14.722626,25.751679,10.355903,19,11.057503,19.313759,8.759075,17,9.010843
4,2021-01,34.478095,23.013375,34.478095,12.629339,25,13.336870,22.985396,9.243457,20,9.276664,17.239047,7.818162,19,8.354041
5,2021-02,35.404057,23.976068,35.404057,12.968519,24,13.202064,23.602705,9.491704,20,9.758460,17.702029,8.028131,19,8.801806
6,2021-03,46.168138,33.514825,46.168138,16.911406,21,17.119622,30.778758,12.377517,18,13.513293,23.084069,10.468965,16,10.994776
7,2021-04,62.670081,47.810157,62.670081,22.956074,19,23.801678,41.780054,16.801630,15,17.103238,31.335041,14.210903,14,15.295641
8,2021-05,85.601268,67.701602,85.601268,31.355776,17,31.451250,57.067512,22.949402,14,24.320720,42.800634,19.410718,13,21.794194
9,2021-06,83.418410,65.628277,83.418410,30.556194,18,32.234269,55.612273,22.364185,14,23.180098,41.709205,18.915739,13,20.726785


In [118]:
payback_results1['Payback Period (2)'].mean()

10.583333333333334

In the worst case, our most recent cohort, the **payback period will shift out by 4 months at an LTV/CAC of 1.5x**, and by **3 for an LTV/CAC of 2x**.

## Conclusions

- A rise in default rate leads to an increas in payback period. This is true for any scenario (1x, 1.5x, 2x). In general, the shift in payback period is worse for lower multipliers (less margin).
- A +1% increase in default rate in the 1.5x scenario will shift the payback period by up to 11 months. At 2x, there is only a shift in payback period of +2 months.
- With a 1.5x scenario, the data suggests we can still maintain a 50mo positive LTV inspite of up to a +1% rise in default rates.